In [ ]:
!unzip "/content/drive/MyDrive/IBM_DATASET/Fertilizers_Recommendation_ System_For_Disease_ Prediction.zip"

: 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1)
x_train=train_datagen.flow_from_directory(r'/content/Dataset Plant Disease/Veg-dataset/Veg-dataset/train_set',target_size=(128,128),batch_size=2,class_mode='categorical')

x_test=test_datagen.flow_from_directory(r'/content/Dataset Plant Disease/Veg-dataset/Veg-dataset/test_set',target_size=(128,128),batch_size=2,class_mode='categorical')






: 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

: 

 **VEGETABLE** **MODEL TRAINING**


In [ ]:
model=Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.summary()


: 

In [ ]:
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))
model.add(Dense(9,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,steps_per_epoch=len(x_train)/2,validation_data=x_test,validation_steps=len(x_test)/2,epochs=3)


: 

# **SAVE MODEL**

In [ ]:
model.save('vegetable.h5')

: 

# **DEPLOY IN CLOUD**

In [ ]:
#savin in tar
!tar -zcvf veg-classification.tgz vegetable.h5

: 

In [ ]:
#IBM DEPLOYMENT

!pip install watson-machine-learning-client




: 

In [ ]:
!pip install ibm_watson_machine_learning

: 

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials ={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"I_ev68NLL5rQXLPxgM9688cVnzrViSxp2L8J2lfQyrTq"

}
client = APIClient(wml_credentials)
client

: 

In [ ]:
client.spaces.get_details()

: 

In [ ]:
def guid_space_name(client,fertilizerrecommedation):
  space=client.spaces.get_details()
  return(next(item for item in space['resources'] if item['entity']['name']==fertilizerrecommedation)['metadata']['id'])

: 

In [29]:
space_uid=guid_space_name(client,'FertilizersRecommedation')
space_uid

'46a4db75-0a5b-4dbc-82e9-6aed90e709c5'

In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [32]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [34]:
model_details=client.repository.store_model(model='veg-classification.tgz',
                                            meta_props={
                                                client.repository.ModelMetaNames.NAME:"VEG CNN model",
                                                client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
                                                client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid


                                            })

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-14T06:29:54.616Z',
  'id': '5bc7565d-65bc-41bb-aea3-5678fe7e5592',
  'modified_at': '2022-11-14T06:30:16.664Z',
  'name': 'Fruit CNN model',
  'owner': 'IBMid-6630043B9T',
  'resource_key': 'c8098545-aa77-4e2d-a765-c8dd6048ca13',
  'space_id': '46a4db75-0a5b-4dbc-82e9-6aed90e709c5'},
 'system': {'warnings': []}}

In [35]:
model_id=client.repository.get_model_id(model_details)
model_id

'2cd4c5d6-152f-47ee-a800-976a08b5b4e8'

In [36]:
client.repository.download(model_id,'veg_model_ibm.tar.gn')

Successfully saved model content to file: 'veg_model_ibm.tar.gn'


'/content/veg_model_ibm.tar.gn'